<a href="https://colab.research.google.com/github/ArjunNair2004/Stock-Prediction/blob/main/ML_Act_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install yfinance
!pip install numpy pandas matplotlib tensorflow
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken 2mkrLdh6TGO3P0H2eNvSBLIEtAX_3fSWj5B9sat7WN8FyjHD


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import streamlit as st

# Function to load and preprocess the stock data
def load_data(ticker):
    data = yf.download(ticker, start='2015-01-01', end='2024-01-01')
    return data[['Close']]

# Function to create the LSTM model
def create_model(X_train):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Function to prepare the data for the LSTM model
def prepare_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    training_data_len = int(np.ceil(len(scaled_data) * 0.8))

    train_data = scaled_data[0:training_data_len, :]
    test_data = scaled_data[training_data_len - 60:, :]

    X_train, y_train = [], []
    for i in range(60, len(train_data)):
        X_train.append(train_data[i - 60:i, 0])
        y_train.append(train_data[i, 0])

    X_test, y_test = [], []
    for i in range(60, len(test_data)):
        X_test.append(test_data[i - 60:i, 0])
        y_test.append(test_data[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_test, y_test = np.array(X_test), np.array(y_test)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    return X_train, y_train, X_test, y_test, scaler

# Function to predict stock prices
def predict_prices(ticker):
    data = load_data(ticker)
    X_train, y_train, X_test, y_test, scaler = prepare_data(data)

    model = create_model(X_train)
    model.fit(X_train, y_train, batch_size=64, epochs=10)

    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    return data, predictions

# Streamlit App Interface
st.title("Stock Price Prediction App")
ticker = st.text_input("Enter stock ticker (e.g., AAPL, GOOGL):", value="AAPL")

if st.button("Predict"):
    data, predictions = predict_prices(ticker)
    st.subheader(f"Stock Price Prediction for {ticker}")

    # Plot the results
    fig, ax = plt.subplots()
    ax.plot(data.index, data['Close'], label='Actual Prices')
    ax.plot(data.index[-len(predictions):], predictions, label='Predicted Prices', linestyle='--')
    plt.xlabel("Date")
    plt.ylabel("Close Price USD ($)")
    plt.title(f"{ticker} Price Prediction")
    plt.legend()
    st.pyplot(fig)


Writing app.py


In [ ]:
from pyngrok import ngrok

# Start ngrok tunnel to the Streamlit app on port 8501
public_url = ngrok.connect(8501, "http")
print(f"Access your Streamlit app here: {public_url}")

# Run the Streamlit app
!streamlit run app.py &>/dev/null&


Access your Streamlit app here: NgrokTunnel: "https://2cc0-35-190-130-2.ngrok-free.app" -> "http://localhost:8501"


NameError: name 'history' is not defined